# Deep Q Network plays Flappy Bird

In this nn assignment we are supposed to train a neural net to play flappy bird. The library we used to simulate the game is flappy-bird-gymnasium (https://github.com/markub3327/flappy-bird-gymnasium).

### Network architecture

The Deep Q network architecture features 3 layers of convolution followed by 2 fully connected layers. The convolutional layers help extract important features from the input image, simplifying the objective of the fully connected layers, since they don't have to worry about exact pixel locations and can focus on more general aspects of the game state (like how close the bird is to a pipe).

In [ ]:
import numpy as np
import torch.nn as nn

class DQNetwork(nn.Module):

    def __init__(self, input_shape, num_actions):
        super(DQNetwork, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=8, stride=4, padding=0),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
        )
        self.fc = nn.Sequential(
            nn.Linear(self._calculate_fc_input(input_shape), 512),
            nn.ReLU(),
            nn.Linear(512, num_actions),
        )

    def _calculate_fc_input(self, input_shape):
        with torch.no_grad():
            dummy_input = torch.zeros(1, *input_shape)
            output = self.conv(dummy_input)
            return int(np.prod(output.size()))

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        print(x.shape)
        return self.fc(x)

### Hyperparameters

* IMAGE_SIZE is the size to which we resize the image in the preprocessing step.
* LEARNING_RATE is the learning rate of the dqn.<br>
* GAMMA is the discount rate for the q learning algorithm.
* BUFFER_SIZE is the maximum size of the replay buffer which is implemented using a deque.
* BATCH_SIZE is the size of a batch that's used when the dqn is updated.
* EPSILON_START is the starting value of epsilon in the implementation of epsilon greedy.
* EPSILON_DECAY is the rate at which epsilon decays (epsilon *= EPSILON_DECAY).
* EPSILON_MIN is the minimum value at which epsilon stops decaying.
* TARGET_UPDATE_FREQ is the frequency at which the target_dqn is updated with the weights of the q_network.
* NUM_EPISODES is the number of runs the algorithm simulates.



In [ ]:
IMAGE_SIZE = (63, 112)
LEARNING_RATE = 1e-4
GAMMA = 0.99
BUFFER_SIZE = 10000
BATCH_SIZE = 64
EPSILON_START = 0.8
EPSILON_DECAY = 0.99
EPSILON_MIN = 0.01
TARGET_UPDATE_FREQ = 100
NUM_EPISODES = 1000

### Image preprocessing
We trained our model on images rather than feature information (pipe positions, bird position etc.) since we wanted to experiment with a convolutional neural network architecture. Thus, we preprocessed the image in order to make the job easier for the dqn. <br>

The methods we implemented are resizing, background removal, converting to grayscale and thresholding.

In [ ]:
import cv2

def preprocess(image: np.ndarray, background: int = 200, threshold: int = 254):
    image[image[:, :, 0] == background] = [255, 255, 255]
    grayscale = 0.2989 * image[..., 0] + 0.5870 * image[..., 1] + 0.1140 * image[..., 2]
    binary_image = (grayscale < threshold).astype(np.uint8)

    return cv2.resize(binary_image, IMAGE_SIZE, interpolation=cv2.INTER_AREA)

The result of the preprocessing for a frame: <br>
![Preprocessed frame](./images/preprocessed_frame.png "Preprocessed frame")

### Replay Buffer
The replay buffer is a deque in which we memorize the description of the last BUFFER_SIZE states simulated by the algorithm.<br>
The sample(self, batch_size) method is used to take a subset of batch_size states from the replay buffer in order to give them to the q_network.


In [ ]:
from collections import deque
import random

class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def add(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        return (
            np.array(states),
            np.array(actions),
            np.array(rewards, dtype=np.float32),
            np.array(next_states),
            np.array(dones, dtype=np.float32),
        )

    def size(self):
        return len(self.buffer)

### The Q learning algorithm
First we initialize the networks used and the replay buffer.<br>
#### The used networks:
* <b>The q_network</b> predicts the q_values of the current state should take given the preprocessed image of it.<br>
* <b>The target_network</b> predicts the q_values for the next state given the preprocessed image of it.<br>
We need to separate them because the q_network needs a fixed function to optimize. For TARGET_UPDATE_FREQ episodes, the target_network remains unchanged to help the q_network with consistent predictions of the q_values of the next state. After TARGET_UPDATE_FREQ episodes, the state of the q_network is copied into the state of the target_network.<br>
If we were to use only one dqn, it would not converge since it would try to optimize a function predicted by itself, which changes everytime the network is updated.<br>
#### Epsilon greedy
The epsilon greedy method gives the model a chance to explore the search space by making a random action with probability of epsilon.<br>
After every episode, the value of epsilon decays by EPSILON_DECAY.<br>
#### Assessing the state of the game
We use the env.step() method provided by the flappy-bird-gymnasium library in order to assess the immediate reward of our action.<br>
We append the state, the action, the reward, the next_state, and whether the game has finished to the replay buffer.
#### Training of the network
1. First, we take a sample of size BATCH_SIZE from the replay buffer.
2. We convert the components of each state tuple to tensors.
3. We approximate the q values of each state in the batch using the q_network.
4. We approximate the maximum q values for each next state using the target_network.
5. We calculate the target values using the formula targets = rewards + (1 - dones) * GAMMA * max_next_q_values. We add the immediate reward of the state with the discounted maximum values for the next states. If the run ends on the next state, we don't add anythong to the reward, hence the (1 - dones).
6. Compute the loss and backpropagate in order to update the q_network.
7. Every TARGET_UPDATE_FREQ frames, we update the target_network's weights with the current network's weights.
8. Decay epsilon by EPSILON_DECAY.

In [ ]:
import torch.optim as optim
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_dql(env):
    # Initialize networks
    input_shape = (1, *IMAGE_SIZE)  # Single grayscale channel
    num_actions = env.action_space.n
    q_network = DQNetwork(input_shape, num_actions).to(device)
    target_network = DQNetwork(input_shape, num_actions).to(device)
    target_network.load_state_dict(q_network.state_dict())
    optimizer = optim.Adam(q_network.parameters(), lr=LEARNING_RATE)
    replay_buffer = ReplayBuffer(BUFFER_SIZE)

    epsilon = EPSILON_START
    total_steps = 0

    for episode in range(NUM_EPISODES):
        env.reset()
        state = preprocess(env.render())
        state = np.expand_dims(state, axis=0)  # Add channel dimension (1, 84, 84)
        total_reward = 0
        done = False

        while not done:
            # Epsilon-greedy action selection
            if random.random() < epsilon:
                action = env.action_space.sample()  # Explore
            else:
                with torch.no_grad():
                    state_tensor = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
                    q_values = q_network(state_tensor)
                    action = torch.argmax(q_values).item()  # Exploit

            # Perform action in the environment
            next_state, reward, done, _, _ = env.step(action)
            next_state = preprocess(env.render())
            next_state = np.expand_dims(next_state, axis=0)

            # if episode >= 590:
            #     print(f"....{reward}")

            # Store transition in replay buffer
            replay_buffer.add(state, action, reward, next_state, done)
            state = next_state
            total_reward += reward

            # Train the network
            if replay_buffer.size() > BATCH_SIZE:
                states, actions, rewards, next_states, dones = replay_buffer.sample(BATCH_SIZE)
                states = torch.tensor(states, dtype=torch.float32, device=device)
                actions = torch.tensor(actions, dtype=torch.long, device=device)
                rewards = torch.tensor(rewards, dtype=torch.float32, device=device)
                next_states = torch.tensor(next_states, dtype=torch.float32, device=device)
                dones = torch.tensor(dones, dtype=torch.float32, device=device)

                # Compute Q-values and targets
                q_values = q_network(states).gather(1, actions.unsqueeze(1)).squeeze(1)
                with torch.no_grad():
                    max_next_q_values = target_network(next_states).max(1)[0]
                    targets = rewards + (1 - dones) * GAMMA * max_next_q_values

                # Compute loss and update Q-network
                loss = nn.MSELoss()(q_values, targets)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            total_steps += 1

            # Update target network periodically
            if total_steps % TARGET_UPDATE_FREQ == 0:
                target_network.load_state_dict(q_network.state_dict())

        # Decay epsilon
        epsilon = max(epsilon * EPSILON_DECAY, EPSILON_MIN)

        print(f"Episode {episode + 1}, Total Reward: {total_reward}, Epsilon: {epsilon:.3f}, Score: {0}")

    env.close()
    return q_network

### Results
We ran the algorithm multiple times, changing the values of some hyperparameters.<br>
| Episodes      | FC layers   | Average score      | Best score      |
|---------------|---------------|---------------|---------------|
| 1000 | 1 (512)  | 12.24  | 52  |
| 1000  |  2 (512x256)  | 12.92  | 75  |
| 10000  |  2 (512x256)  | 3.24  | 17  |
